## Segmenting and Clustering Assignment

In [3]:
#Import libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd

### I'm scraping the wiki page for the table and pulling out the rows and columns

In [4]:
beautiful = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(beautiful.content, 'lxml')
table = soup.find('table', class_="wikitable sortable")
table_rows = table.find_all('tr')
table_cols = table.find_all('th')

print(table_cols)

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
</th>]


### Putting the rows into a dataframe

In [5]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighbourhood"])

### Cleaning the dataframe

In [6]:
df=df.drop(index=0)
# Removing unassigned boroughs
df = df.drop(df[(df.Borough == "Not assigned")].index)
# Removing extra \n characters from the web scraping
df.Neighbourhood = df.Neighbourhood.str.replace(pat = "\n", repl = "", regex=False)
df.reset_index(drop=True, inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [7]:
# If the Neighbourhood is empty but the Borough isn't then assign the Borough name to the Neighbourhood

for i in df.index:
    if df.Neighbourhood[i] == "Not assigned":
        df.Neighbourhood[i] = df.Borough[i]


### Condensing the dataframe by grouping all Neighbourhoods into Boroughs and Postcodes

In [8]:
df_sorted = df.sort_values('Postcode')
df_sorted.reset_index(drop=True, inplace=True)

for i in df_sorted.index:
    if df_sorted.Postcode[i] == df_sorted.Postcode[i+1] and df_sorted.Borough[i] == df_sorted.Borough[i+1]:
        df_sorted.Neighbourhood[i+1] = df_sorted.Neighbourhood[i] + ", " + df_sorted.Neighbourhood[i+1]
        df_sorted.drop([i], axis=0, inplace=True)
        
# It throws an error but does the job

KeyError: 210

In [9]:
df_sorted.reset_index(drop=True, inplace=True)

df_sorted.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df_sorted.shape

(103, 3)

In [11]:
#the geocoder didn't work for me so I'm using the csv file instead

link = "https://cocl.us/Geospatial_data"

geo_coords = pd.read_csv(link)
geo_coords = geo_coords.rename(columns={'Postal Code': 'Postcode'})
geo_coords.head(10)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
#adding the data into a new dataframe

geo_df = df_sorted.merge(geo_coords,on='Postcode',how='left')

geo_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [13]:
# Using Toronto as the center of the map

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [15]:
neighbourhood_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Toronto


# add Toronto as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Toronto',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(neighbourhood_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(geo_df.Latitude, geo_df.Longitude, geo_df.Neighbourhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(neighbourhood_map)

# display map
neighbourhood_map